In [1]:
] precompile

Precompiling project...
┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [2]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions

In [86]:
# @testset "radial map" begin
    γ = 1.0
    λ = 0.2
    δ = 1e-4
    κ = 5.0

    U = RadialMap(4,2, γ=γ, λ=λ, δ=δ, κ=κ)
    @test size(U) == (4,2)

    @test U.γ==1.0
    @test U.λ==0.2
    @test U.δ==1e-4
    @test U.κ==5.0

    for i=1:4
        @test size(U.U[i]) ==(i,2)
    end


    U = RadialMap(2,2)
    U.U[1].a[1] .= ones(2+3)
    U.U[2].a[1] .= 5*ones(2+1)
    U.U[2].a[2] .= ones(2+3)


    U11 = RadialMapComponent(1,2, [zeros(4)], [ones(4)], [ones(2+3)])
    U22 = RadialMapComponent(2,2, [zeros(2),zeros(4)], [ones(2), ones(4)], [5*ones(2+1), ones(2+3)])
    
    x = randn()
    y = randn(2)
    z = randn(2)

    @test isapprox(U([x; x]), [U11([x]); U22([x; x])], atol = 1e-10)

    @test isapprox(U(y), [U11(y[1]); U22(y)], atol = 1e-10)
    @test isapprox(U(z), [U11(z[1]); U22(z)], atol = 1e-10)

    U = RadialMapComponent(2,2)
    a2 = rand(5)
    U.a[2] .= a2

    
    @test isapprox(ForwardDiff.gradient(x->U(x), y)[2], 
                   ψ₀′(0.0, a2[1])(y[1])+ rbf(0.0, a2[2])(y[1]) + rbf(0.0, a2[3])(y[1]) +
                   ψpp1′(0.0, a2[4])(y[2]), atol = 1e-10)
# end

Test Failed at In[86]:43
  Expression: isapprox((ForwardDiff.gradient((x->(U(x);)), y))[2], (ψ₀′(0.0, a2[1]))(y[1]) + (rbf(0.0, a2[2]))(y[1]) + (rbf(0.0, a2[3]))(y[1]) + (ψpp1′(0.0, a2[4]))(y[2]), atol = 1.0e-10)
   Evaluated: isapprox(1.0259005301753914, 0.6497606317219393; atol = 1.0e-10)


LoadError: [91mThere was an error during testing[39m

In [81]:
ForwardDiff.gradient(x->U(x), y)[2]

1.1133613986456734

In [82]:
 ψ₀′(0.0, a2[1])(y[1])

0.6864603124227382

In [19]:
@testset "RadialMapComponent (Nx,p) = (1,1)" begin

    C = RadialMapComponent(1,1)

    @test C.Nx == 1
    @test C.p == 1

    @test size(C.ξ,1) ==1
    @test size(C.ξ[1],1) ==1+2

    @test size(C.σ,1) ==1
    @test size(C.σ[1],1) ==1+2

    @test size(C.ξ,1) ==1
    @test size(C.ξ[1],1) ==1+2

    @test size(C.a,1) ==1
    @test size(C.a[1],1) ==1+3

    C.a[1] .= [1.0; 2.0; 3.0; 1.0]

    x = randn()
    @test abs(C(x) - (1.0 + 2.0*ψ₀(0.0,1.0)(x) +
                     3.0*ψj(0.0,1.0)(x) + 1.0*ψpp1(0.0,1.0)(x)))< 1e-10

    @test abs(C([x]) - (1.0 + 2.0*ψ₀(0.0,1.0)(x) +
                      3.0*ψj(0.0,1.0)(x) + 1.0*ψpp1(0.0,1.0)(x)))< 1e-10

end


Test Summary:                     | Pass  Total
RadialMapComponent (Nx,p) = (1,1) |   12     12


Test.DefaultTestSet("RadialMapComponent (Nx,p) = (1,1)", Any[], 12, false)

In [5]:
@testset "Validate loss function I" begin
    Nx = 1
    Ne = 10
    p = 3
    γ = 2.0
    
    X = reshape([-1.5438
                 -1.5518
                    0.8671
                   -0.1454
                   -0.3862
                    1.3162
                   -0.7965
                    0.1354
                    0.4178
                    0.8199],1,Ne)
    
    S = RadialMap(Nx, p; γ = γ);
    center_std(S, X);

    W = create_weights(S, X)
    weights(S, X, W)

    ψ_off, ψ_mono, dψ_mono = rearrange_ricardo(W,Nx);
    μψ = deepcopy(mean(ψ_mono, dims=2))
    σψ = deepcopy(std(ψ_mono, dims=2, corrected=false))

    ψ_mono .-= μψ
    ψ_mono ./= σψ
    dψ_mono ./= σψ

    A = ψ_mono*ψ_mono'

# Value for this example from Ricardo Baptista's code
#     A = [1.000000000000000   0.980746250237442   0.949585186065365   0.874893160310884   0.682688353791794
#    0.980746250237442   1.000000000000000   0.992392810306380   0.951686939442048   0.794587369107029
#    0.949585186065365   0.992392810306380   1.000000000000000   0.979987807512526   0.841920530108716
#    0.874893160310884   0.951686939442048   0.979987807512526   1.000000000000000   0.924572967646111
#    0.682688353791794   0.794587369107029   0.841920530108716   0.924572967646111   1.000000000000000]
#
#    dψ = [1.943027194236598   0.904099317171178   0.363590761264482   0.103708119866970   0.003658358886509
#    1.949127329018457   0.900154645724251   0.358540918818191   0.101474664830914   0.003524731655464
#    0.436105078812650   0.785902678977702   0.903915317182112   1.478321127032274   2.769938074690495
#    0.940754438483933   1.187522900065327   1.375388815415954   1.249642233914406   0.493692853837867
#    1.094596689101933   1.214822387670764   1.280911634416390   0.981040834765123   0.264471303061320
#    0.288722974980864   0.555375334050147   0.517307925073611   1.026368388570445   4.005813485764689
#    1.380134162445589   1.181212729079411   0.975263367972602   0.543176897811297   0.074249159004658
#    0.776788122078680   1.114885036778955   1.375388815415954   1.502313102175688   0.916745911337340
#    0.630159061208850   1.005528754335045   1.256866405570924   1.625364312645948   1.525991525022805
#    0.454238308718550   0.810349920313954   0.945882762308085   1.512212666353956   2.631844256908664]
    nb = size(ψ_mono,1)
    rmul!(A,1/Ne)
    λ = 0.0
    δ = 0.0;

    x0 = ones(nb);

    Lhd = LHD(A, Matrix(dψ_mono'), λ, δ)
    Lhd(x0,true)

    @test norm(Lhd.J[1] - 9.858183469194048)<1e-10
    @test norm(Lhd.G - [4.254136703227439;4.518792556597960;4.583786234498148;4.551417068663062;4.037990186069983])<1e-10
    @test norm(Lhd.H - [1.092420543831142   1.038440852206475   0.988737431456852   0.902158085870462   0.699932284218600;
   1.038440852206475   1.044839328780384   1.030133800308906   0.984518683331781   0.822101516960292;
   0.988737431456852   1.030133800308906   1.036827933644060   1.015461469462408   0.872825798615600;
   0.902158085870462   0.984518683331781   1.015461469462408   1.039520711149700   0.969205731345725;
   0.699932284218600   0.822101516960292   0.872825798615600   0.969205731345725   1.085482924097099])<1e-10

   J = Lhd(x0,false;noutput =1)
   @test norm(J[1] - 9.858183469194048)<1e-10

   J, G = Lhd(x0,false;noutput =2)
   @test norm(J[1] - 9.858183469194048)<1e-10
   @test norm(G - [4.254136703227439;4.518792556597960;4.583786234498148;4.551417068663062;4.037990186069983])<1e-10

   J, G, H = Lhd(x0,false;noutput =3)
   @test norm(J[1] - 9.858183469194048)<1e-10
   @test norm(G - [4.254136703227439;4.518792556597960;4.583786234498148;4.551417068663062;4.037990186069983])<1e-10
   @test norm(H - [1.092420543831142   1.038440852206475   0.988737431456852   0.902158085870462   0.699932284218600;
  1.038440852206475   1.044839328780384   1.030133800308906   0.984518683331781   0.822101516960292;
  0.988737431456852   1.030133800308906   1.036827933644060   1.015461469462408   0.872825798615600;
  0.902158085870462   0.984518683331781   1.015461469462408   1.039520711149700   0.969205731345725;
  0.699932284218600   0.822101516960292   0.872825798615600   0.969205731345725   1.085482924097099])<1e-10

end

Validate loss function I: Error During Test at In[5]:1
  Got exception outside of a @test
  UndefVarError: RadialMap not defined
  Stacktrace:
   [1] top-level scope at In[5]:18
   [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1115
   [3] top-level scope at In[5]:2
   [4] include_string(::Function, ::Module, ::String, ::String) at ./loading.jl:1091
   [5] softscope_include_string(::Module, ::String, ::String) at /home/mat/.julia/packages/SoftGlobalScope/u4UzH/src/SoftGlobalScope.jl:65
   [6] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/mat/.julia/packages/IJulia/e8kqU/src/execute_request.jl:67
   [7] #invokelatest#1 at ./essentials.jl:710 [inlined]
   [8] invokelatest at ./essentials.jl:709 [inlined]
   [9] eventloop(::ZMQ.Socket) at /home/mat/.julia/packages/IJulia/e8kqU/src/eventloop.jl:8
   [10] (::IJulia.var"#15#18")() at ./task.jl:356
  
Test Summary:            | Error  Total
Validate loss function I |     1 

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.[39m